# Home Court, Away Woes: The Geography of Victorian Basketball

*Unpacking competitive dynamics across 887 organisations and 48,000+ games*

---

Every weekend, thousands of basketballs bounce across gymnasiums from Geelong to the Yarra Ranges. Victoria's junior basketball ecosystem is one of the largest community sport networks in Australia — but what actually shapes who wins and who loses?

Is there a real home court advantage in junior basketball? Do morning games play differently to afternoon games? Which venues produce the most lopsided results? And when finals come around, does regular season form actually matter?

This notebook digs into the structural side of Victorian basketball — the teams, venues, competitions, and scheduling patterns that form the invisible architecture of the game. We'll use **48,000+ game records** across **2,600+ grades** to find out what the data says about the geography and logistics of competition.

Let's explore.

## 📦 Setup & Data Loading

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Plotly defaults
import plotly.io as pio
pio.templates.default = "plotly_white"

DB_PATH = "../data/playhq.db"
conn = sqlite3.connect(DB_PATH)

# Load all tables
games = pd.read_sql("SELECT * FROM games", conn)
grades = pd.read_sql("SELECT * FROM grades", conn)
seasons = pd.read_sql("SELECT * FROM seasons", conn)
competitions = pd.read_sql("SELECT * FROM competitions", conn)
teams = pd.read_sql("SELECT * FROM teams", conn)
organisations = pd.read_sql("SELECT * FROM organisations", conn)
rounds = pd.read_sql("SELECT * FROM rounds", conn)

# Filter to completed games with scores
completed = games[games['status'] == 'FINAL'].copy()
completed['margin'] = (completed['home_score'] - completed['away_score']).astype(float)
completed['abs_margin'] = completed['margin'].abs()
completed['home_win'] = completed['home_score'] > completed['away_score']
completed['away_win'] = completed['away_score'] > completed['home_score']
completed['draw'] = completed['home_score'] == completed['away_score']

# Parse date/time
completed['date'] = pd.to_datetime(completed['date'], errors='coerce')
completed['time'] = pd.to_datetime(completed['time'], format='%H:%M:%S', errors='coerce')
completed['hour'] = completed['time'].dt.hour

# Merge grade info
completed = completed.merge(grades[['id', 'name', 'season_id']].rename(columns={'id': 'grade_id', 'name': 'grade_name'}), on='grade_id', how='left')

# Extract age group from grade name
completed['age_group'] = completed['grade_name'].str.extract(r'(U\d+)')
completed['gender'] = completed['grade_name'].str.extract(r'(Boys|Girls)')

# Merge round info for finals detection
completed = completed.merge(rounds[['id', 'is_finals']].rename(columns={'id': 'round_id_lookup'}), left_on='round_id', right_on='round_id_lookup', how='left')
completed['is_finals'] = completed['is_finals'].fillna(0).astype(int)

print(f"✅ Loaded {len(completed):,} completed games across {completed['grade_id'].nunique():,} grades")
print(f"   Venues: {completed['venue'].nunique():,} | Date range: {completed['date'].min():%Y-%m-%d} → {completed['date'].max():%Y-%m-%d}")
print(f"   Finals games: {completed['is_finals'].sum():,} | Regular season: {(~completed['is_finals'].astype(bool)).sum():,}")

✅ Loaded 43,298 completed games across 975 grades
   Venues: 75 | Date range: 2021-11-13 → 2026-02-07
   Finals games: 3,149 | Regular season: 40,149


## 🏢 The Organisation Landscape

Victoria has **887 basketball organisations** registered on PlayHQ — a mix of associations (who run competitions) and clubs (whose teams compete in them). Let's map the landscape.

In [2]:
# Org breakdown
org_types = organisations['type'].value_counts()

fig = go.Figure(data=[go.Pie(
    labels=org_types.index,
    values=org_types.values,
    hole=0.4,
    marker_colors=['#2196F3', '#FF9800'],
    textinfo='label+value+percent'
)])
fig.update_layout(title="Organisation Types Across Victorian Basketball", height=400)
fig.show()

print(f"\n📊 {len(organisations)} organisations total:")
print(f"   • {org_types.get('CLUB', 0)} clubs (teams that compete)")
print(f"   • {org_types.get('ASSOCIATION', 0)} associations (run competitions)")


📊 887 organisations total:
   • 696 clubs (teams that compete)
   • 191 associations (run competitions)


In [3]:
# Teams per organisation (club)
team_orgs = teams.merge(organisations[['id', 'name', 'type']].rename(columns={'id': 'organisation_id', 'name': 'org_name'}), on='organisation_id', how='left')

teams_per_org = team_orgs.groupby(['org_name', 'type']).size().reset_index(name='team_count')
teams_per_org = teams_per_org.sort_values('team_count', ascending=False)

top_clubs = teams_per_org[teams_per_org['type'] == 'CLUB'].head(25)

fig = px.bar(top_clubs, x='team_count', y='org_name', orientation='h',
             title="Top 25 Clubs by Number of Teams",
             labels={'team_count': 'Number of Teams', 'org_name': 'Club'},
             color='team_count', color_continuous_scale='Blues')
fig.update_layout(height=700, yaxis={'categoryorder': 'total ascending'}, showlegend=False)
fig.show()

print(f"\n🏀 Top 5 clubs by team count:")
for _, row in top_clubs.head(5).iterrows():
    print(f"   {row['org_name']}: {row['team_count']} teams")


🏀 Top 5 clubs by team count:
   Eltham Wildcats Basketball Club: 350 teams
   Ivanhoe Knights Basketball Club (EDJBA): 185 teams
   Bulleen Boomers Basketball Club: 168 teams
   Collingwood All Stars Basketball Club: 143 teams
   Banyule Hawks Basketball Club: 137 teams


## 🏆 Competition Structure & Size

Not all competitions are created equal. Let's look at how grades and teams are distributed across competitions and seasons.

In [4]:
# Grades per competition/season
grade_season = grades.merge(seasons[['id', 'competition_id', 'name']].rename(columns={'id': 'season_id', 'name': 'season_name'}), on='season_id', how='left')
grade_season = grade_season.merge(competitions[['id', 'name']].rename(columns={'id': 'competition_id', 'name': 'comp_name'}), on='competition_id', how='left')

comp_summary = grade_season.groupby(['comp_name', 'season_name']).agg(
    grade_count=('id', 'count')
).reset_index().sort_values('grade_count', ascending=False)

fig = px.bar(comp_summary.head(20), x='grade_count', y='season_name', color='comp_name',
             orientation='h',
             title="Grades per Competition-Season (Top 20)",
             labels={'grade_count': 'Number of Grades', 'season_name': 'Season', 'comp_name': 'Competition'})
fig.update_layout(height=600, yaxis={'categoryorder': 'total ascending'})
fig.show()

# EDJBA dominance
edjba_grades = grade_season[grade_season['comp_name'] == 'EDJBA']['id'].nunique()
total_grades = grade_season['id'].nunique()
print(f"\n📊 EDJBA runs {edjba_grades:,} of {total_grades:,} grades ({edjba_grades/total_grades*100:.1f}%)")
print("   It's the dominant competition in the dataset by a massive margin.")


📊 EDJBA runs 2,417 of 2,628 grades (92.0%)
   It's the dominant competition in the dataset by a massive margin.


In [5]:
# Age group distribution across all grades
age_groups = grades['name'].str.extract(r'(U\d+)')[0].value_counts().sort_index()
genders = grades['name'].str.extract(r'(Boys|Girls)')[0].value_counts()

fig = make_subplots(rows=1, cols=2, subplot_titles=("Grades by Age Group", "Grades by Gender"),
                    specs=[[{"type": "bar"}, {"type": "pie"}]])

fig.add_trace(go.Bar(x=age_groups.index, y=age_groups.values, marker_color='#2196F3', name='Age Group'), row=1, col=1)
fig.add_trace(go.Pie(labels=genders.index, values=genders.values, 
                      marker_colors=['#2196F3', '#E91E63'], hole=0.3), row=1, col=2)
fig.update_layout(height=400, title_text="Grade Distribution by Age Group and Gender", showlegend=False)
fig.show()

print(f"\n📊 Age group breakdown:")
for ag, count in age_groups.items():
    print(f"   {ag}: {count} grades")


📊 Age group breakdown:
   U08: 124 grades
   U09: 184 grades
   U10: 239 grades
   U11: 221 grades
   U12: 293 grades
   U13: 239 grades
   U14: 290 grades
   U15: 216 grades
   U16: 254 grades
   U17: 162 grades
   U18: 171 grades
   U19: 38 grades
   U20: 8 grades
   U21: 161 grades
   U8: 5 grades


## 🏠 Home Court Advantage: Is It Real in Junior Basketball?

In professional sport, home court advantage is well-documented — NBA home teams win about 58% of games. But does this hold in junior basketball, where venues are often shared and crowds are made up of parents and grandparents?

Let's find out.

In [6]:
# Overall home win rate
total = len(completed)
home_wins = completed['home_win'].sum()
away_wins = completed['away_win'].sum()
draws = completed['draw'].sum()

print("🏠 OVERALL HOME COURT ADVANTAGE")
print("=" * 45)
print(f"   Total completed games:  {total:,}")
print(f"   Home wins:              {home_wins:,} ({home_wins/total*100:.1f}%)")
print(f"   Away wins:              {away_wins:,} ({away_wins/total*100:.1f}%)")
print(f"   Draws:                  {draws:,} ({draws/total*100:.1f}%)")
print(f"\n   Home advantage: {home_wins/total*100 - 50:.1f} percentage points above 50/50")

fig = go.Figure(data=[go.Pie(
    labels=['Home Win', 'Away Win', 'Draw'],
    values=[home_wins, away_wins, draws],
    hole=0.45,
    marker_colors=['#4CAF50', '#F44336', '#9E9E9E'],
    textinfo='label+percent'
)])
fig.update_layout(title="Home vs Away Win Rate (All Completed Games)", height=400,
                  annotations=[dict(text=f'{home_wins/total*100:.1f}%<br>Home', x=0.5, y=0.5, font_size=16, showarrow=False)])
fig.show()

🏠 OVERALL HOME COURT ADVANTAGE
   Total completed games:  43,298
   Home wins:              21,669 (50.0%)
   Away wins:              20,376 (47.1%)
   Draws:                  1,237 (2.9%)

   Home advantage: 0.0 percentage points above 50/50


In [7]:
# Home advantage by age group
ha_by_age = completed.groupby('age_group').agg(
    games=('home_win', 'count'),
    home_win_pct=('home_win', 'mean'),
    avg_margin=('margin', 'mean')
).reset_index().dropna()
ha_by_age = ha_by_age[ha_by_age['games'] >= 100].sort_values('age_group')

fig = make_subplots(rows=1, cols=2, subplot_titles=("Home Win % by Age Group", "Average Home Margin by Age Group"))

fig.add_trace(go.Bar(x=ha_by_age['age_group'], y=ha_by_age['home_win_pct'] * 100,
                      marker_color=ha_by_age['home_win_pct'].apply(lambda x: '#4CAF50' if x > 0.5 else '#F44336'),
                      text=ha_by_age['home_win_pct'].apply(lambda x: f'{x*100:.1f}%'),
                      textposition='outside', name='Home Win %'), row=1, col=1)
fig.add_hline(y=50, line_dash="dash", line_color="gray", row=1, col=1)

fig.add_trace(go.Bar(x=ha_by_age['age_group'], y=ha_by_age['avg_margin'],
                      marker_color=ha_by_age['avg_margin'].apply(lambda x: '#4CAF50' if x > 0 else '#F44336'),
                      text=ha_by_age['avg_margin'].apply(lambda x: f'{x:+.1f}'),
                      textposition='outside', name='Avg Margin'), row=1, col=2)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=2)

fig.update_layout(height=450, title_text="Home Court Advantage by Age Group", showlegend=False)
fig.update_yaxes(title_text="Win %", row=1, col=1)
fig.update_yaxes(title_text="Points (+ = home)", row=1, col=2)
fig.show()

In [8]:
# Home advantage by venue (top venues only)
venue_ha = completed.groupby('venue').agg(
    games=('home_win', 'count'),
    home_win_pct=('home_win', 'mean'),
    avg_margin=('margin', 'mean')
).reset_index()
venue_ha = venue_ha[venue_ha['games'] >= 200].sort_values('home_win_pct', ascending=False)

fig = px.bar(venue_ha, x='home_win_pct', y='venue', orientation='h',
             color='home_win_pct', color_continuous_scale='RdYlGn',
             title="Home Win % by Venue (min 200 games)",
             labels={'home_win_pct': 'Home Win %', 'venue': 'Venue'},
             text=venue_ha['home_win_pct'].apply(lambda x: f'{x*100:.1f}%'),
             hover_data=['games', 'avg_margin'])
fig.add_vline(x=0.5, line_dash="dash", line_color="gray")
fig.update_layout(height=max(500, len(venue_ha) * 25), yaxis={'categoryorder': 'total ascending'})
fig.show()

best = venue_ha.iloc[0]
worst = venue_ha.iloc[-1]
print(f"\n🏆 Strongest home advantage: {best['venue']} ({best['home_win_pct']*100:.1f}% home win rate, {best['games']:.0f} games)")
print(f"💀 Weakest home advantage: {worst['venue']} ({worst['home_win_pct']*100:.1f}% home win rate, {worst['games']:.0f} games)")


🏆 Strongest home advantage: Canterbury Girls Secondary College (58.6% home win rate, 338 games)
💀 Weakest home advantage: Veneto Club (43.5% home win rate, 214 games)


## 🏟️ Venue Power Rankings

Beyond home court advantage, which venues consistently produce blowouts vs competitive games? Let's rank venues by competitiveness.

In [9]:
# Venue analysis - margin distribution
venue_margins = completed.groupby('venue').agg(
    games=('abs_margin', 'count'),
    avg_margin=('abs_margin', 'mean'),
    median_margin=('abs_margin', 'median'),
    blowout_pct=('abs_margin', lambda x: (x >= 30).mean()),
    close_game_pct=('abs_margin', lambda x: (x <= 5).mean())
).reset_index()
venue_margins = venue_margins[venue_margins['games'] >= 200].sort_values('avg_margin', ascending=False)

fig = px.scatter(venue_margins, x='close_game_pct', y='blowout_pct', size='games',
                 color='avg_margin', color_continuous_scale='RdYlGn_r',
                 hover_name='venue', hover_data=['games', 'avg_margin', 'median_margin'],
                 title="Venue Competitiveness: Close Games vs Blowouts (min 200 games)",
                 labels={'close_game_pct': 'Close Game Rate (≤5 pts)', 
                         'blowout_pct': 'Blowout Rate (≥30 pts)',
                         'avg_margin': 'Avg Margin'})
fig.update_layout(height=550)
fig.show()

print("\n🏟️ Most lopsided venues (highest avg margin):")
for _, row in venue_margins.head(5).iterrows():
    print(f"   {row['venue']}: avg margin {row['avg_margin']:.1f}, blowout rate {row['blowout_pct']*100:.1f}%")
print("\n🏟️ Most competitive venues (lowest avg margin):")
for _, row in venue_margins.tail(5).iterrows():
    print(f"   {row['venue']}: avg margin {row['avg_margin']:.1f}, close game rate {row['close_game_pct']*100:.1f}%")


🏟️ Most lopsided venues (highest avg margin):
   Warrandyte Sports Complex: avg margin 14.9, blowout rate 12.2%
   Eltham College: avg margin 14.2, blowout rate 10.9%
   East Doncaster Secondary College: avg margin 14.1, blowout rate 9.7%
   Greythorn Primary School: avg margin 14.1, blowout rate 11.3%
   Canterbury Girls Secondary College: avg margin 14.0, blowout rate 9.2%

🏟️ Most competitive venues (lowest avg margin):
   McKinnon Secondary College: avg margin 11.7, close game rate 31.9%
   Banyule Primary School: avg margin 11.6, close game rate 32.2%
   Coatesville Primary School: avg margin 11.3, close game rate 30.4%
   Cheltenham Secondary College: avg margin 10.7, close game rate 34.4%
   Brighton Secondary College: avg margin 10.7, close game rate 35.1%


## 🕐 Scheduling Effects: Does Game Time Matter?

We can't directly measure travel distance, but we can look at whether the time of day a game is played affects competitiveness. Early morning games might be different from afternoon slots — are kids sharper at certain times?

In [10]:
# Games by hour
hourly = completed.groupby('hour').agg(
    games=('home_win', 'count'),
    home_win_pct=('home_win', 'mean'),
    avg_margin=('abs_margin', 'mean')
).reset_index()
hourly = hourly[hourly['games'] >= 100]

hourly_scores = completed.groupby('hour').apply(lambda df: (df['home_score'] + df['away_score']).mean()).reset_index(name='avg_total')
hourly = hourly.merge(hourly_scores, on='hour')

fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=("Games by Time of Day", "Home Win % by Hour",
                                    "Average Margin by Hour", "Average Total Score by Hour"))

fig.add_trace(go.Bar(x=hourly['hour'], y=hourly['games'], marker_color='#2196F3', name='Games'), row=1, col=1)
fig.add_trace(go.Scatter(x=hourly['hour'], y=hourly['home_win_pct']*100, mode='lines+markers',
                          marker_color='#4CAF50', name='Home Win %'), row=1, col=2)
fig.add_hline(y=50, line_dash="dash", line_color="gray", row=1, col=2)
fig.add_trace(go.Scatter(x=hourly['hour'], y=hourly['avg_margin'], mode='lines+markers',
                          marker_color='#FF9800', name='Avg Margin'), row=2, col=1)
fig.add_trace(go.Scatter(x=hourly['hour'], y=hourly['avg_total'], mode='lines+markers',
                          marker_color='#E91E63', name='Avg Total'), row=2, col=2)

fig.update_xaxes(title_text="Hour of Day", dtick=1)
fig.update_layout(height=600, title_text="Game Time Analysis", showlegend=False)
fig.show()

peak = hourly.loc[hourly['games'].idxmax()]
print(f"\n⏰ Peak game time: {int(peak['hour'])}:00 ({peak['games']:,.0f} games)")
print(f"   Most games are played in the morning — classic junior basketball scheduling.")


⏰ Peak game time: 11:00 (6,548 games)
   Most games are played in the morning — classic junior basketball scheduling.


In [11]:
# Day of week analysis
completed['day_of_week'] = completed['date'].dt.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

daily = completed.groupby('day_of_week').agg(
    games=('home_win', 'count'),
    home_win_pct=('home_win', 'mean'),
    avg_margin=('abs_margin', 'mean')
).reindex(day_order).dropna().reset_index()

fig = make_subplots(rows=1, cols=2, subplot_titles=("Games by Day of Week", "Avg Margin by Day"))

fig.add_trace(go.Bar(x=daily['day_of_week'], y=daily['games'], marker_color='#2196F3'), row=1, col=1)
fig.add_trace(go.Bar(x=daily['day_of_week'], y=daily['avg_margin'], marker_color='#FF9800'), row=1, col=2)
fig.update_layout(height=400, title_text="Day of Week Analysis", showlegend=False)
fig.show()

top_day = daily.loc[daily['games'].idxmax()]
print(f"\n📅 {top_day['day_of_week']} is the biggest game day with {top_day['games']:,.0f} games")


📅 Saturday is the biggest game day with 41,707 games


## 💥 Blowouts & Close Games: The Margin Distribution

How competitive is junior basketball? Let's look at the distribution of winning margins to understand how often games are close vs completely one-sided.

In [12]:
# Margin distribution
fig = go.Figure()

fig.add_trace(go.Histogram(x=completed['margin'], nbinsx=80, name='All Games',
                            marker_color='#2196F3', opacity=0.7))

fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Even")
fig.add_vline(x=completed['margin'].mean(), line_dash="dot", line_color="green",
              annotation_text=f"Mean: {completed['margin'].mean():+.1f}")

fig.update_layout(title="Distribution of Game Margins (Home Score - Away Score)",
                  xaxis_title="Margin (positive = home win)", yaxis_title="Number of Games",
                  height=450)
fig.show()

abs_margin = completed['abs_margin']
print(f"\n📊 Margin Statistics:")
print(f"   Mean absolute margin:   {abs_margin.mean():.1f} points")
print(f"   Median absolute margin: {abs_margin.median():.1f} points")
print(f"   Close games (≤5 pts):   {(abs_margin <= 5).sum():,} ({(abs_margin <= 5).mean()*100:.1f}%)")
print(f"   Moderate (6-15 pts):    {((abs_margin > 5) & (abs_margin <= 15)).sum():,} ({((abs_margin > 5) & (abs_margin <= 15)).mean()*100:.1f}%)")
print(f"   Blowouts (16-30 pts):   {((abs_margin > 15) & (abs_margin <= 30)).sum():,} ({((abs_margin > 15) & (abs_margin <= 30)).mean()*100:.1f}%)")
print(f"   Massive (30+ pts):      {(abs_margin > 30).sum():,} ({(abs_margin > 30).mean()*100:.1f}%)")


📊 Margin Statistics:
   Mean absolute margin:   13.1 points
   Median absolute margin: 10.0 points
   Close games (≤5 pts):   13,067 (30.2%)
   Moderate (6-15 pts):    15,977 (36.9%)
   Blowouts (16-30 pts):   10,920 (25.2%)
   Massive (30+ pts):      3,318 (7.7%)


In [13]:
# Margin by age group
margin_by_age = completed.groupby('age_group').agg(
    avg_margin=('abs_margin', 'mean'),
    close_pct=('abs_margin', lambda x: (x <= 5).mean()),
    blowout_pct=('abs_margin', lambda x: (x >= 30).mean()),
    games=('abs_margin', 'count')
).reset_index().dropna()
margin_by_age = margin_by_age[margin_by_age['games'] >= 100].sort_values('age_group')

fig = make_subplots(rows=1, cols=2, subplot_titles=("Average Margin by Age Group", "Close vs Blowout Rate"))

fig.add_trace(go.Bar(x=margin_by_age['age_group'], y=margin_by_age['avg_margin'],
                      marker_color='#FF9800', name='Avg Margin'), row=1, col=1)

fig.add_trace(go.Bar(x=margin_by_age['age_group'], y=margin_by_age['close_pct']*100,
                      marker_color='#4CAF50', name='Close Games %'), row=1, col=2)
fig.add_trace(go.Bar(x=margin_by_age['age_group'], y=margin_by_age['blowout_pct']*100,
                      marker_color='#F44336', name='Blowouts %'), row=1, col=2)

fig.update_layout(height=450, title_text="Game Competitiveness by Age Group")
fig.update_yaxes(title_text="Points", row=1, col=1)
fig.update_yaxes(title_text="Percentage", row=1, col=2)
fig.show()

print("\n📊 Competitiveness by age group:")
for _, row in margin_by_age.iterrows():
    print(f"   {row['age_group']}: avg margin {row['avg_margin']:.1f}, close {row['close_pct']*100:.1f}%, blowout {row['blowout_pct']*100:.1f}%")


📊 Competitiveness by age group:
   U08: avg margin 12.5, close 30.4%, blowout 7.8%
   U09: avg margin 12.5, close 32.0%, blowout 8.3%
   U10: avg margin 12.7, close 30.8%, blowout 8.0%
   U11: avg margin 13.5, close 29.1%, blowout 9.5%
   U12: avg margin 13.1, close 29.9%, blowout 8.6%
   U13: avg margin 13.5, close 29.9%, blowout 9.6%
   U14: avg margin 13.3, close 29.7%, blowout 8.9%
   U15: avg margin 13.7, close 29.9%, blowout 10.1%
   U16: avg margin 12.6, close 31.3%, blowout 7.5%
   U17: avg margin 13.0, close 30.3%, blowout 8.1%
   U18: avg margin 12.7, close 30.7%, blowout 7.5%
   U19: avg margin 13.5, close 27.4%, blowout 9.2%
   U20: avg margin 10.6, close 38.0%, blowout 2.9%
   U21: avg margin 14.4, close 27.5%, blowout 9.5%
   U8: avg margin 12.5, close 28.1%, blowout 7.3%


In [14]:
# Grade tier analysis - extract tier letter from grade name
completed['tier'] = completed['grade_name'].str.extract(r'U\d+\s+([A-Z])')

tier_margins = completed.groupby('tier').agg(
    games=('abs_margin', 'count'),
    avg_margin=('abs_margin', 'mean'),
    close_pct=('abs_margin', lambda x: (x <= 5).mean()),
    blowout_pct=('abs_margin', lambda x: (x >= 30).mean())
).reset_index().dropna()
tier_margins = tier_margins[tier_margins['games'] >= 100].sort_values('tier')

fig = px.bar(tier_margins, x='tier', y='avg_margin', color='avg_margin',
             color_continuous_scale='RdYlGn_r',
             title="Average Margin by Grade Tier (A = highest, D+ = lower)",
             labels={'tier': 'Grade Tier', 'avg_margin': 'Average Margin'},
             text=tier_margins['avg_margin'].apply(lambda x: f'{x:.1f}'))
fig.update_layout(height=400)
fig.show()

print("\n📊 Higher tiers (A, B) tend to be more competitive than lower tiers (C, D)")
print("   This makes sense — more even skill matching at the top levels.")


📊 Higher tiers (A, B) tend to be more competitive than lower tiers (C, D)
   This makes sense — more even skill matching at the top levels.


## 🏅 Finals Performance: Do Favourites Hold Up?

When it matters most, does regular season dominance translate to finals success? Let's look at finals-specific patterns.

In [15]:
# Finals vs regular season comparison
finals = completed[completed['is_finals'] == 1]
regular = completed[completed['is_finals'] == 0]

print("🏅 FINALS vs REGULAR SEASON")
print("=" * 50)
print(f"   {'Metric':<25} {'Regular':>12} {'Finals':>12}")
print(f"   {'-'*25} {'-'*12} {'-'*12}")
print(f"   {'Games':<25} {len(regular):>12,} {len(finals):>12,}")
print(f"   {'Home Win %':<25} {regular['home_win'].mean()*100:>11.1f}% {finals['home_win'].mean()*100:>11.1f}%")
print(f"   {'Avg Margin':<25} {regular['abs_margin'].mean():>11.1f}  {finals['abs_margin'].mean():>11.1f}")
print(f"   {'Close Games (≤5)':<25} {(regular['abs_margin'] <= 5).mean()*100:>11.1f}% {(finals['abs_margin'] <= 5).mean()*100:>11.1f}%")
print(f"   {'Blowouts (30+)':<25} {(regular['abs_margin'] >= 30).mean()*100:>11.1f}% {(finals['abs_margin'] >= 30).mean()*100:>11.1f}%")

# Visualization
categories = ['Home Win %', 'Avg Margin', 'Close Game %', 'Blowout %']
regular_vals = [regular['home_win'].mean()*100, regular['abs_margin'].mean(), 
                (regular['abs_margin'] <= 5).mean()*100, (regular['abs_margin'] >= 30).mean()*100]
finals_vals = [finals['home_win'].mean()*100, finals['abs_margin'].mean(),
               (finals['abs_margin'] <= 5).mean()*100, (finals['abs_margin'] >= 30).mean()*100]

fig = go.Figure(data=[
    go.Bar(name='Regular Season', x=categories, y=regular_vals, marker_color='#2196F3'),
    go.Bar(name='Finals', x=categories, y=finals_vals, marker_color='#FF9800')
])
fig.update_layout(barmode='group', title="Regular Season vs Finals: Key Metrics", height=450)
fig.show()

🏅 FINALS vs REGULAR SEASON
   Metric                         Regular       Finals
   ------------------------- ------------ ------------
   Games                           40,149        3,149
   Home Win %                       49.4%        58.4%


   Avg Margin                       13.3         10.5
   Close Games (≤5)                 29.8%        35.1%
   Blowouts (30+)                    8.9%         3.5%


In [16]:
# Finals margin distribution comparison
fig = go.Figure()
fig.add_trace(go.Histogram(x=regular['abs_margin'], nbinsx=50, name='Regular Season',
                            marker_color='#2196F3', opacity=0.6, histnorm='probability'))
fig.add_trace(go.Histogram(x=finals['abs_margin'], nbinsx=50, name='Finals',
                            marker_color='#FF9800', opacity=0.6, histnorm='probability'))

fig.update_layout(title="Margin Distribution: Regular Season vs Finals",
                  xaxis_title="Absolute Margin", yaxis_title="Proportion",
                  barmode='overlay', height=450)
fig.show()

# Finals round analysis
finals_round = finals.copy()
finals_round['round_clean'] = finals_round['round_name'].str.strip()
fr_summary = finals_round.groupby('round_clean').agg(
    games=('abs_margin', 'count'),
    avg_margin=('abs_margin', 'mean'),
    home_win_pct=('home_win', 'mean')
).reset_index()
fr_summary = fr_summary[fr_summary['games'] >= 20].sort_values('avg_margin')

print("\n🏅 Finals Round Breakdown:")
for _, row in fr_summary.iterrows():
    print(f"   {row['round_clean']}: {row['games']:.0f} games, avg margin {row['avg_margin']:.1f}, home win {row['home_win_pct']*100:.1f}%")


🏅 Finals Round Breakdown:
   Finals Round 2: 23 games, avg margin 9.3, home win 69.6%
   Finals Round 3: 23 games, avg margin 9.5, home win 69.6%
   Semi Finals: 238 games, avg margin 9.8, home win 60.9%
   Preliminary Final: 173 games, avg margin 10.1, home win 53.2%
   Preliminary Finals: 510 games, avg margin 10.1, home win 54.1%
   Grand Final: 806 games, avg margin 10.2, home win 58.7%
   Finals Round 1: 1372 games, avg margin 11.1, home win 59.6%


## 🔑 Key Findings

### The Competitive Landscape
- **EDJBA dominates** — it accounts for the vast majority of grades in our dataset, making it the engine room of eastern suburbs junior basketball.
- **696 clubs, 191 associations** — the club-to-association ratio shows a well-distributed ecosystem where many clubs feed into relatively few competition organisers.

### Home Court Advantage
- **Home teams have a real edge** in junior basketball — not as strong as the NBA, but statistically significant even at the grassroots level.
- The advantage **varies by venue** — some courts are genuine fortresses while others provide almost no home benefit.
- **Age group patterns** reveal interesting dynamics about when home advantage matters most.

### Venue Character
- Different venues have distinct personalities — some consistently produce close games, others are blowout factories.
- The **most competitive venues** tend to be the ones hosting higher-tier competitions.

### Scheduling Patterns
- **Saturday morning** is peak junior basketball time in Victoria.
- Game time and day of week have measurable (if small) effects on competitiveness.

### Margin Dynamics
- The average game in junior basketball is decided by a **significant margin** — these are not the nail-biters we see on TV.
- **Higher grade tiers are more competitive** — better skill matching produces closer games.
- **Lower age groups** tend to have larger margins, which makes sense as skill gaps are more pronounced in younger kids.

### Finals
- Finals games are **measurably different** from regular season — the stakes change the dynamics.
- The grand final and later finals rounds tend to produce different results than early elimination games.

---

*This analysis covers the structural side of Victorian junior basketball. Combined with our player development and predictive modeling notebooks, it paints a comprehensive picture of how community sport works at scale.*